In [ ]:
#https://trackmytrading.wordpress.com/
!pip install backtrader pandas_datareader
!pip install pyfolio


In [ ]:
import pandas_datareader.data as web
import pyfolio as pf
import yfinance as yf
import backtrader as bt
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import backtrader.analyzers as btanalyzers
import backtrader.strategies as btstrats


In [ ]:


# Download data for a single stock
data_df = yf.download('IBM', start='2022-01-01', end='2023-04-10')
data_df.head()



In [ ]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    analyzed_map = {}
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    analyzed_map = {"total_open": total_open ,"total_closed": total_closed ,"total_won": total_won ,"total_lost": total_lost ,
                    "win_streak": win_streak ,"lose_streak": lose_streak ,"pnl_net": pnl_net ,"strike_rate": strike_rate }

    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))
    
    return analyzed_map
 
def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))
    return sqn

In [ ]:
# Strategy: Base class for user-defined strategies
# BuyAndHold: Buy and hold strategy
# CMA: Centered Moving Average strategy
# CrossOver: Cross-over strategy
# Dummy: Dummy strategy
# ElliotWave: Elliot Wave strategy
# GoldenCross: Golden Cross strategy
# Ichimoku: Ichimoku Kinko Hyo strategy
# MACD: Moving Average Convergence Divergence strategy
# MAE: Maximum Adverse Excursion strategy
# Momentum: Momentum strategy
# MovingAverage: Moving Average strategy
# RSI: Relative Strength Index strategy
# Stochastic: Stochastic strategy
# WilliamsR: Williams %R strategy
# ZigZag: Zig Zag strategy

In [ ]:
#-- strategies
import backtrader as bt
import backtrader.talib as talib

class MACDStrategy(bt.Strategy):
    params = (
        ('fast_length', 12),
        ('slow_length', 26),
        ('signal_length', 9),
    )
    
    def __init__(self):
        self.macd = bt.indicators.MACD(
            self.data,
            period_me1=self.p.fast_length,
            period_me2=self.p.slow_length,
            period_signal=self.p.signal_length,
        )

    def next(self):
        if not self.position:
            if self.macd.macd[0] > self.macd.signal[0]:
                self.buy()
        elif self.macd.macd[0] < self.macd.signal[0]:
            self.sell()



class MovingAverageCrossStrategy(bt.Strategy):
    
    params = (('pfast', 10), ('pslow', 30),)
    
    def __init__(self):
        self.fastma = bt.indicators.SMA(self.data, period=self.params.pfast)
        self.slowma = bt.indicators.SMA(self.data, period=self.params.pslow)
        self.crossover = bt.indicators.CrossOver(self.fastma, self.slowma)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()



class DonchianChannelStrategy(bt.Strategy):
    params = (('period', 20),)

    def __init__(self):
        self.donchian = talib.DONCHIAN(self.data.high, self.data.low, self.data.close, timeperiod=self.p.period)

    def next(self):
        if self.donchian.lines.dcl[0] < self.data.close[0]:
            self.buy()
        elif self.donchian.lines.dch[0] > self.data.close[0]:
            self.sell()

class ParabolicSARStrategy(bt.Strategy):

    params = (('af', 0.02), ('amax', 0.2))

    def __init__(self):
        self.sar = bt.indicators.PSAR(self.data, af=self.params.af, afmax=self.params.amax)

    def next(self):
        if not self.position:
            if self.data.close[0] > self.sar[0]:
                self.buy()
        elif self.data.close[0] < self.sar[0]:
            self.close()



class MovingAverageEnvelopeStrategy(bt.Strategy):

    params = (('p', 20), ('dev', 0.05),)

    def __init__(self):
        self.ma = bt.indicators.SMA(self.data, period=self.params.p)
        self.upper = self.ma * (1.0 + self.params.dev)
        self.lower = self.ma * (1.0 - self.params.dev)

    def next(self):
        if not self.position:
            if self.data.close[0] > self.upper[0]:
                self.buy()
        elif self.data.close[0] < self.lower[0]:
            self.close()






In [ ]:
def analyze_result(result,initial_cash):
  firstStrat = result

 
  # print the analyzers
  analyzed_data = printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
  analyzed_data["sqn"] = printSQN(firstStrat.analyzers.sqn.get_analysis())
  analyzed_data["sharperatio"] = result.analyzers.sharpe_ratio.get_analysis()['sharperatio']
  analyzed_data["max_drawdown"] = result.analyzers.drawdown.get_analysis()['max']['drawdown']
  analyzed_data["pnl"] = cerebro.broker.getvalue() - initial_cash

  print(analyzed_data)
  sharpe_ratio =result.analyzers.sharpe_ratio.get_analysis()
  print("Sharpe Ratio:", sharpe_ratio['sharperatio'])

  drawdown = result.analyzers.drawdown.get_analysis()
  print("Max Drawdown:", drawdown['max']['drawdown'])
  
  #Get final portfolio Value
  portvalue = cerebro.broker.getvalue()
  print("final and last port value:  "+str(portvalue))
  pnl = portvalue - initial_cash
  #Print out the final result
  print('Final Portfolio Value: ${}'.format(portvalue))
  print('P/L: ${}'.format(pnl))


In [ ]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2022, 4, 18)
data = bt.feeds.PandasData(dataname=data_df)

In [ ]:

def init_cerebro(data, cash , comm):
  cerebro = bt.Cerebro()
  # Add analyzers
  #cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
  # cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')
  # cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
  # cerebro.addobserver(bt.observers.Value)
  # cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
  # cerebro.addanalyzer(bt.analyzers.Returns)
  # cerebro.addanalyzer(bt.analyzers.DrawDown)

  cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
  cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
  cerebro.broker.setcash(cash)
  cerebro.broker.setcommission(commission=comm)



  cerebro.adddata(data)
  return cerebro



In [ ]:
cerebro = init_cerebro(data)
cerebro.addstrategy(MovingAverageEnvelopeStrategy)
#cerebro.addstrategy(btstrats.SMA_CrossOver)
cerebro.addstrategy(MACDStrategy)
#cerebro.addstrategy(ParabolicSARStrategy)
cerebro.strats.pop()

results = cerebro.run()
print(results) 

In [ ]:
strategyList = [MovingAverageEnvelopeStrategy,MACDStrategy,ParabolicSARStrategy,btstrats.SMA_CrossOver ] 


In [ ]:
cash = 10000
comm = 0.003

for strtgy in strategyList:
  strtgy_name = strtgy.__name__
  print(strtgy_name)
  cerebro = init_cerebro(data,cash,comm)
  cerebro.addstrategy(strtgy)
  res = cerebro.run()
  chart_file = 'result'+strtgy_name+'.png'
  cerebro.plot()[0][0].savefig(chart_file, dpi=300)
  returns_result = analyze_result(res[0],cash)
  cerebro.strats.pop()


MovingAverageEnvelopeStrategy
Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               0              3              1              2              
               Strike Rate    Win Streak     Losing Streak  PnL Net        
               33.333333333333331              2              -18.06         
SQN: -0.87
{'total_open': 0, 'total_closed': 3, 'total_won': 1, 'total_lost': 2, 'win_streak': 1, 'lose_streak': 2, 'pnl_net': -18.06, 'strike_rate': 33.33333333333333, 'sqn': -0.87, 'sharperatio': -12.077136214229057, 'max_drawdown': 0.3291442005940251, 'pnl': -18.055208145142387}
Sharpe Ratio: -12.077136214229057
Max Drawdown: 0.3291442005940251
final and last port value:  9981.944791854858
Final Portfolio Value: $9981.944791854858
P/L: $-18.055208145142387
MACDStrategy
Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               1              10             2              8  